In [13]:
# NVIDIA Merlin 기반 안전한 CTR 데이터 분석
# RTX 3090 24GB 최적화 버전

import cupy as cp
import os
import gc
import time
import warnings
warnings.filterwarnings('ignore')


In [14]:

print("🚀 NVIDIA Merlin CTR 분석 시작")
print("=" * 60)

# 1. 환경 확인 및 라이브러리 import
def check_environment():
    """환경 및 라이브러리 확인"""
    print("🔍 환경 확인 중...")
    
    required_libs = {
        'cudf': '23.10+',
        'cupy': '12.0+', 
        'nvtabular': '23.08+',
        'pandas': '1.5+',
        'pyarrow': '12.0+'
    }
    
    available_libs = {}
    
    # CuDF 확인
    try:
        import cudf
        available_libs['cudf'] = cudf.__version__
        print(f"✅ cuDF {cudf.__version__} 사용 가능")
    except ImportError:
        print("❌ cuDF 없음 - GPU 가속 불가")
        available_libs['cudf'] = None
    
    # CuPy 확인  
    try:
        import cupy as cp
        available_libs['cupy'] = cp.__version__
        print(f"✅ CuPy {cp.__version__} 사용 가능")
        
        # GPU 메모리 확인
        mempool = cp.get_default_memory_pool()
        print(f"🎯 GPU 메모리 풀 초기화 완료")
    except ImportError:
        print("❌ CuPy 없음 - GPU 메모리 관리 제한")
        available_libs['cupy'] = None
    
    # NVTabular 확인
    try:
        import nvtabular as nvt
        available_libs['nvtabular'] = nvt.__version__
        print(f"✅ NVTabular {nvt.__version__} 사용 가능")
    except ImportError:
        print("❌ NVTabular 없음 - Merlin 기능 불가")
        available_libs['nvtabular'] = None
    
    # 기본 라이브러리들
    try:
        import pandas as pd
        import pyarrow.parquet as pq
        import numpy as np
        available_libs['pandas'] = pd.__version__
        available_libs['pyarrow'] = pq.__version__ if hasattr(pq, '__version__') else 'available'
        print(f"✅ Pandas {pd.__version__}, PyArrow 사용 가능")
    except ImportError as e:
        print(f"❌ 기본 라이브러리 오류: {e}")
    
    return available_libs


🚀 NVIDIA Merlin CTR 분석 시작


In [15]:

# 환경 확인 실행
libs = check_environment()


🔍 환경 확인 중...
✅ cuDF 23.10.02 사용 가능
✅ CuPy 13.6.0 사용 가능
🎯 GPU 메모리 풀 초기화 완료
❌ NVTabular 없음 - Merlin 기능 불가
✅ Pandas 1.5.3, PyArrow 사용 가능


In [16]:

# 2. GPU 메모리 관리 함수들
def print_gpu_memory():
    """GPU 메모리 사용량 출력"""
    try:
        import cupy as cp
        mempool = cp.get_default_memory_pool()
        used = mempool.used_bytes() / 1024**3
        total = mempool.total_bytes() / 1024**3 if mempool.total_bytes() > 0 else 24.0  # RTX 3090 기준
        print(f"🎯 GPU 메모리: {used:.1f}GB / {total:.1f}GB")
        return used, total
    except:
        print("🎯 GPU 메모리 정보 확인 불가")
        return 0, 0

def clear_gpu_memory():
    """GPU 메모리 정리"""
    try:
        import cupy as cp
        mempool = cp.get_default_memory_pool()
        mempool.free_all_blocks()
        gc.collect()
        print("🧹 GPU 메모리 정리 완료")
    except:
        gc.collect()
        print("🧹 CPU 메모리 정리 완료")


In [17]:

# 초기 메모리 상태 확인
print_gpu_memory()


🎯 GPU 메모리: 0.0GB / 24.0GB


(0.0, 24.0)

In [18]:

# 3. 파일 기본 정보 확인 (메모리 안전)
def safe_file_info(file_path):
    """파일 정보를 안전하게 확인"""
    try:
        import pyarrow.parquet as pq
        
        print(f"📁 파일 분석: {file_path}")
        
        # 파일 존재 확인
        if not os.path.exists(file_path):
            print(f"❌ 파일이 존재하지 않습니다: {file_path}")
            return None
        
        # 파케이 파일 메타데이터만 읽기
        pf = pq.ParquetFile(file_path)
        
        file_info = {
            'rows': pf.metadata.num_rows,
            'columns': len(pf.schema),
            'file_size_gb': os.path.getsize(file_path) / 1024**3,
            'row_groups': pf.num_row_groups,
            'column_names': pf.schema.names
        }
        
        print(f"   📊 행 수: {file_info['rows']:,}")
        print(f"   📊 컬럼 수: {file_info['columns']}")
        print(f"   📊 파일 크기: {file_info['file_size_gb']:.2f} GB")
        print(f"   📊 Row Groups: {file_info['row_groups']}")
        
        return file_info
        
    except Exception as e:
        print(f"❌ 파일 정보 확인 실패: {e}")
        return None

# 파일 정보 확인
train_info = safe_file_info('../data/train.parquet')
test_info = safe_file_info('../data/test.parquet')

if not train_info:
    print("⚠️ train.parquet 파일 경로를 확인해주세요")
    print("현재 디렉토리:", os.getcwd())
    print("파일 목록:", [f for f in os.listdir('.') if f.endswith('.parquet')])


📁 파일 분석: ../data/train.parquet
   📊 행 수: 10,704,179
   📊 컬럼 수: 119
   📊 파일 크기: 8.19 GB
   📊 Row Groups: 1
📁 파일 분석: ../data/test.parquet
   📊 행 수: 1,527,298
   📊 컬럼 수: 119
   📊 파일 크기: 1.20 GB
   📊 Row Groups: 1


In [19]:

# 4. CuDF 기반 안전한 샘플 분석
def cudf_safe_sample_analysis(file_path, sample_size=50000):
    """CuDF를 사용한 안전한 샘플 분석"""
    
    if not libs.get('cudf'):
        print("❌ CuDF가 설치되지 않아 GPU 분석 불가")
        return fallback_pandas_analysis(file_path, sample_size)
    
    try:
        import cudf
        import cupy as cp
        
        print(f"🎯 CuDF 기반 샘플 분석 시작 (샘플 크기: {sample_size:,})")
        
        clear_gpu_memory()
        start_time = time.time()
        
        # 작은 청크로 시작
        chunk_size = min(sample_size, 10000)
        print(f"   청크 크기: {chunk_size:,}행")
        
        # 파케이 파일에서 첫 번째 row group 읽기
        import pyarrow.parquet as pq
        pf = pq.ParquetFile(file_path)
        
        # 첫 번째 청크를 pandas로 읽고 cudf로 변환
        first_chunk_pd = pf.read_row_group(0, columns=None).to_pandas()
        
        # 샘플 크기에 맞게 자르기
        if len(first_chunk_pd) > sample_size:
            first_chunk_pd = first_chunk_pd.sample(n=sample_size, random_state=42)
        
        print(f"   Pandas 청크 로드 완료: {len(first_chunk_pd):,}행")
        
        # CuDF로 변환
        sample_gdf = cudf.from_pandas(first_chunk_pd)
        print(f"   CuDF 변환 완료")
        
        # GPU에서 기본 분석 수행
        print(f"🔍 GPU 분석 결과:")
        print(f"   📏 데이터 형태: {sample_gdf.shape}")
        
        # 타겟 변수 분석
        if 'clicked' in sample_gdf.columns:
            click_counts = sample_gdf['clicked'].value_counts().to_pandas()
            click_rate = sample_gdf['clicked'].mean()
            print(f"   🎯 클릭률: {click_rate:.4f} ({click_rate*100:.2f}%)")
            print(f"   🎯 클릭 분포: {click_counts.to_dict()}")
        
        # 데이터 타입 분석
        print(f"   📊 데이터 타입별 컬럼 수:")
        dtype_info = {}
        for col in sample_gdf.columns:
            dtype = str(sample_gdf[col].dtype)
            dtype_info[dtype] = dtype_info.get(dtype, 0) + 1
        
        for dtype, count in dtype_info.items():
            print(f"      {dtype}: {count}개")
        
        # 결측값 확인
        null_counts = sample_gdf.isnull().sum().to_pandas()
        null_cols = null_counts[null_counts > 0]
        print(f"   ❓ 결측값 있는 컬럼: {len(null_cols)}개")
        
        # 카디널리티 분석 (일부 컬럼만)
        print(f"   🎲 카디널리티 분석 (상위 10개):")
        cardinality_info = {}
        for col in sample_gdf.columns[:20]:  # 처음 20개 컬럼만
            try:
                unique_count = sample_gdf[col].nunique()
                cardinality_info[col] = unique_count
            except:
                pass
        
        # 카디널리티 내림차순 정렬
        sorted_cardinality = sorted(cardinality_info.items(), key=lambda x: x[1], reverse=True)
        for col, count in sorted_cardinality[:10]:
            print(f"      {col}: {count}")
        
        # 처리 시간 출력
        elapsed = time.time() - start_time
        print(f"   ⏱️ 처리 시간: {elapsed:.2f}초")
        
        # 메모리 정리
        del sample_gdf, first_chunk_pd
        clear_gpu_memory()
        
        return True
        
    except Exception as e:
        print(f"❌ CuDF 분석 실패: {e}")
        clear_gpu_memory()
        return fallback_pandas_analysis(file_path, sample_size)


In [20]:

def fallback_pandas_analysis(file_path, sample_size=10000):
    """CuDF 실패시 Pandas 폴백"""
    try:
        import pandas as pd
        import pyarrow.parquet as pq
        
        print(f"🔄 Pandas 폴백 모드 (샘플 크기: {sample_size:,})")
        
        # 매우 작은 샘플만 로드
        pf = pq.ParquetFile(file_path)
        
        # 첫 번째 row group에서 소량만 읽기
        sample_df = pf.read_row_group(0).to_pandas()
        
        if len(sample_df) > sample_size:
            sample_df = sample_df.sample(n=sample_size, random_state=42)
        
        print(f"📊 기본 정보:")
        print(f"   형태: {sample_df.shape}")
        
        if 'clicked' in sample_df.columns:
            click_rate = sample_df['clicked'].mean()
            print(f"   클릭률: {click_rate:.4f}")
        
        print(f"   데이터 타입: {sample_df.dtypes.value_counts().to_dict()}")
        
        # 메모리 정리
        del sample_df
        gc.collect()
        
        return True
        
    except Exception as e:
        print(f"❌ Pandas 분석도 실패: {e}")
        return False


In [21]:

# 5. NVTabular 기반 전처리 테스트
def test_nvtabular_workflow():
    """NVTabular 워크플로우 테스트"""
    
    if not libs.get('nvtabular'):
        print("❌ NVTabular 없음 - 고급 전처리 불가")
        return False
    
    try:
        import nvtabular as nvt
        from nvtabular import ops
        from merlin.io import Dataset
        
        print("🔧 NVTabular 워크플로우 테스트")
        
        # 간단한 워크플로우 생성 테스트
        # 실제 데이터 없이 구조만 확인
        
        # 카테고리 컬럼 정의 (일반적인 CTR 데이터 기준)
        categorical_columns = ['gender', 'age_group', 'inventory_id', 'day_of_week', 'hour']
        
        # 연속형 컬럼 (예시)
        continuous_columns = [f'feat_a_{i}' for i in range(1, 10)]
        
        # 워크플로우 정의
        cat_workflow = categorical_columns >> ops.Categorify()
        cont_workflow = continuous_columns >> ops.FillMissing() >> ops.Normalize()
        
        workflow = nvt.Workflow(cat_workflow + cont_workflow)
        
        print("✅ NVTabular 워크플로우 생성 성공")
        print(f"   범주형 컬럼: {len(categorical_columns)}개")
        print(f"   연속형 컬럼: {len(continuous_columns)}개")
        
        return True
        
    except Exception as e:
        print(f"❌ NVTabular 테스트 실패: {e}")
        return False


In [26]:

# 6. 실행부
if __name__ == "__main__":
    print("\n" + "="*60)
    print("🎯 메인 분석 시작")
    print("="*60)
    
    # GPU 메모리 초기 상태
    print_gpu_memory()
    
    # 파일이 존재하는 경우에만 분석 진행
    if train_info:
        print(f"\n📊 Train 데이터 분석 시작")
        success = cudf_safe_sample_analysis('../data/train.parquet', sample_size=20000)
        
        if success:
            print("✅ 샘플 분석 완료")
        else:
            print("❌ 분석 실패 - 환경 설정을 확인해주세요")
    
    # NVTabular 테스트
    print(f"\n🔧 NVTabular 환경 테스트")
    nvt_success = test_nvtabular_workflow()
    
    # 최종 권장사항
    print(f"\n💡 환경별 권장사항:")
    
    if libs.get('cudf') and libs.get('nvtabular'):
        print("🚀 완전한 Merlin 환경 - 최적 성능 가능")
        print("   1. 전체 데이터 GPU 처리 가능")
        print("   2. Out-of-core 처리로 메모리 안전")
        print("   3. XGBoost GPU 가속 추천")
    elif libs.get('cudf'):
        print("⚡ CuDF 환경 - GPU 가속 가능")
        print("   1. DataFrame 연산 10-50배 가속")
        print("   2. 메모리 효율적 처리")
        print("   3. 추가 NVTabular 설치 권장")
    else:
        print("🐌 기본 환경 - CPU 기반 처리")
        print("   1. 청크 단위 처리 필수")
        print("   2. 샘플링 기반 분석")
        print("   3. Merlin 환경 구축 강력 권장")
    
    print(f"\n🛠️ Merlin 완전 설치 가이드:")
    print("conda create -n merlin python=3.10")
    print("conda activate merlin")
    print("conda install -c rapidsai -c nvidia -c conda-forge cudf=23.10 cuml")
    print("pip install merlin-models nvtabular")
    print("pip install xgboost lightgbm --upgrade")
    
    # 최종 메모리 정리
    clear_gpu_memory()
    print_gpu_memory()
    
    print("\n✅ 분석 완료!")


🎯 메인 분석 시작
🎯 GPU 메모리: 0.0GB / 24.0GB (0.0%)

📊 Train 데이터 분석 시작
🎯 CuDF 기반 샘플 분석 시작 (샘플 크기: 20,000)
🧹 GPU 메모리 정리 완료
   청크 크기: 10,000행
   Pandas 청크 로드 완료: 20,000행
   CuDF 변환 완료
🔍 GPU 분석 결과:
   📏 데이터 형태: (20000, 119)
   🎯 클릭률: 0.0191 (1.91%)
   🎯 클릭 분포: {0: 19619, 1: 381}
   📊 데이터 타입별 컬럼 수:
      object: 6개
      float32: 112개
      int32: 1개
   ❓ 결측값 있는 컬럼: 96개
   🎲 카디널리티 분석 (상위 10개):
      seq: 19832
      l_feat_12: 2120
      l_feat_11: 1233
      l_feat_14: 1178
      l_feat_5: 897
      l_feat_6: 754
      l_feat_9: 443
      l_feat_7: 291
      l_feat_10: 247
      l_feat_4: 26
   ⏱️ 처리 시간: 89.23초
🧹 GPU 메모리 정리 완료
✅ 샘플 분석 완료

🔧 NVTabular 환경 테스트
❌ NVTabular 없음 - 고급 전처리 불가

💡 환경별 권장사항:
⚡ CuDF 환경 - GPU 가속 가능
   1. DataFrame 연산 10-50배 가속
   2. 메모리 효율적 처리
   3. 추가 NVTabular 설치 권장

🛠️ Merlin 완전 설치 가이드:
conda create -n merlin python=3.10
conda activate merlin
conda install -c rapidsai -c nvidia -c conda-forge cudf=23.10 cuml
pip install merlin-models nvtabular
pip install xgboost lightgbm 